In [1]:
## Spell checker from Peter Norvig. Not perfect, but works.
import re, collections

def words(text): return re.findall('[a-z]+', text.lower()) 

def train(features):
    model = collections.defaultdict(lambda: 1)
    for f in features:
        model[f] += 1
    return model

NWORDS = train(words(open('C:/Users/kenndanielso/Documents/Github/mcnulty_yelp/data/big.txt').read()))

alphabet = 'abcdefghijklmnopqrstuvwxyz'

def edits1(word):
   splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
   deletes    = [a + b[1:] for a, b in splits if b]
   transposes = [a + b[1] + b[0] + b[2:] for a, b in splits if len(b)>1]
   replaces   = [a + c + b[1:] for a, b in splits for c in alphabet if b]
   inserts    = [a + c + b     for a, b in splits for c in alphabet]
   return set(deletes + transposes + replaces + inserts)

def known_edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)

def known(words): return set(w for w in words if w in NWORDS)

def correct(word):
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    return max(candidates, key=NWORDS.get)

In [120]:
## Import all modules needed
## lem is lemmatizing the words
from nltk.corpus import wordnet
def lem(x):
    sentence = ''
    for x in re.sub('[^a-z0-9 ]','',x).split():   
        sentence = sentence + ' ' + lemmatizer.lemmatize(correct(x))
    count = 0
    return sentence

import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem import WordNetLemmatizer
%matplotlib inline
from nltk import word_tokenize
class LemmaTokenizer(object):
     def __init__(self):
         self.wnl = WordNetLemmatizer()
     def __call__(self, doc):
         return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

from nltk.corpus import stopwords

stops = set(stopwords.words('english'))

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = CountVectorizer(min_df=10,stop_words=stops,max_features=1000,ngram_range=(1,3),tokenizer=LemmaTokenizer())
tfvectorizer = TfidfVectorizer(stop_words=stops,min_df=10,max_features=1000,ngram_range=(1,3),tokenizer=LemmaTokenizer())

### Take note of the assumptions made in the vectorizer specifications. There are two kinds of vectorizers initialized: count and tfidf. I've limited both to consider only n-grams that appear at least ten times. I've also limited the feature set into the top 1,000 n-grams that appear the most often in the reviews. Also it only extracts unigrams to trigrams. You can edit any of the parameters.

### You can switch between count and tfidf vectorizers by changing between "vectorizer" and "tfvectorizer" in one of the cells below

In [121]:
import pandas as pd
import pickle
with open("C:/Users/kenndanielso/Documents/Github/mcnulty_yelp/data/review_business_df.pkl", 'rb') as picklefile: 
    review_business_df = pickle.load(picklefile)

In [3]:
sample_df = review_business_df.sample(10000,random_state=1).dropna().reset_index(drop=True)

### Note that I've only extract 10,000 sample reviews. This is just to manage its tracktability. Lemmatizing takes a really long time. Indirectly, 10,000 reviews is also big enough relative to the 1,000 features (discussed above).

In [6]:
short_df = sample_df.iloc[:,0:2]

In [7]:
short_df['spell'] = short_df['text'].apply(lambda x: x.lower())

In [8]:
short_df['s_spell_lem'] = short_df['spell'].apply(lem)

In [101]:
new_df = short_df[['s_spell_lem','stars_x']]

In [102]:
corp = list(new_df.s_spell_lem)

### You can change the vectorizer type in the cell below

In [122]:
feature = tfvectorizer.fit_transform(corp)

In [139]:
## Turns count/tfidf matrix into a dataframe
featuredf = pd.DataFrame(feature.A, columns=tfvectorizer.get_feature_names())

In [140]:
## Concat Y and X
new_df = pd.concat((new_df,featuredf),axis=1)

In [141]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 3002 entries, s_spell_lem to young
dtypes: float64(3000), object(2)
memory usage: 229.0+ MB


In [142]:
def string(x):
    if x == 5:
        return "five"
    elif x == 4:
        return 'four'
    elif x == 3:
        return 'three'
    elif x == 2:
        return 'two'
    elif x == 1:
        return 'one'

In [146]:
new_df['stars_x'] = short_df['stars_x'].apply(string)

In [147]:
new_df.head()

,s_spell_lem,stars_x,100,able,absolutely,ache,across,actually,add,added,...,would definitely,would recommend,wrap,wrong,year,yelp,yes,yet,york,young
0,i go out to eat often and have been to many p...,four,0.0,0.0,0.0,0.0,0.126137,0.103447,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,always happysatisfied when i leave they have ...,five,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,i dont like fried chicken there i said it sou...,five,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,love this little hidden gem best go by far in...,five,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,a bit conflict about this cafe because last n...,three,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.062492,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [145]:
import pickle
with open('C:/Users/kenndanielso/Documents/Github/mcnulty_yelp/data/kenn_review_df.pkl', 'wb') as picklefile: 
    pickle.dump(new_df, picklefile)

# Model testing

In [131]:
##Split into train and test at 75/25
from sklearn.cross_validation import train_test_split 
train, test = train_test_split(new_df,test_size = 0.25)

In [132]:
##Split X & Y
X_train = train.iloc[:,2:]
Y_train = train.iloc[:,1]
X_test = test.iloc[:,2:]
Y_test = test.iloc[:,1]

In [133]:
from sklearn.metrics import confusion_matrix

## NaiveBayes
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB().fit(X_train,Y_train)
nb_Y_pred = nb.predict(X_test)
print("NB Accuracy: ",np.mean(nb_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,nb_Y_pred,labels=['one','two','three','four','five']))

NB Accuracy:  0.542
[[145  35  23  25  21]
 [ 54  50  59  54  17]
 [ 27  25 100 174  46]
 [  6   5  40 414 315]
 [ 14   3  19 183 646]]


In [134]:
## Logistic
from sklearn.linear_model import LogisticRegression
log = LogisticRegression().fit(X_train,Y_train)
log_Y_pred = log.predict(X_test)
print("Logistc Accuracy: ",np.mean(log_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,log_Y_pred,labels=['one','two','three','four','five']))

Logistc Accuracy:  0.5372
[[152  32  18  22  25]
 [ 43  59  52  53  27]
 [ 17  30 116 150  59]
 [  2  12  61 370 335]
 [ 14   2  18 185 646]]


In [135]:
## Linear SVC
from sklearn.svm import SVC
svcl = SVC(kernel='linear').fit(X_train,Y_train)
svcl_Y_pred = svcl.predict(X_test)
print("SVC Linear Accuracy: ",np.mean(svcl_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,svcl_Y_pred,labels=['one','two','three','four','five']))

SVC Linear Accuracy:  0.5368
[[143  50  17  20  19]
 [ 47  79  47  44  17]
 [ 22  43 125 130  52]
 [  8  17  69 377 309]
 [ 16   6  30 195 618]]


In [136]:
## RBF SVC
from sklearn.svm import SVC
svcrbf = SVC(kernel='rbf', gamma=1).fit(X_train,Y_train)
svcrbf_Y_pred = svcrbf.predict(X_test)
print("SVC RBF Accuracy: ",np.mean(svcrbf_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,svcrbf_Y_pred,labels=['one','two','three','four','five']))

SVC RBF Accuracy:  0.5104
[[110  14  10  59  56]
 [ 27  22  41 102  42]
 [  8   9  61 227  67]
 [  1   3  12 403 361]
 [  4   1   2 178 680]]


In [137]:
## Decision Tree
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier().fit(X_train,Y_train)
dt_Y_pred = dt.predict(X_test)
print("Decision Tree Accuracy: ",np.mean(dt_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,dt_Y_pred,labels=['one','two','three','four','five']))

Decision Tree Accuracy:  0.3784
[[ 92  36  40  43  38]
 [ 38  38  54  59  45]
 [ 37  61  85 106  83]
 [ 42  48 115 265 310]
 [ 42  30  74 253 466]]


In [138]:
## Random Forests
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier().fit(X_train,Y_train)
rf_Y_pred = rf.predict(X_test)
print("Random Forests Accuracy: ",np.mean(rf_Y_pred == np.array(Y_test)))
print(confusion_matrix(Y_test,rf_Y_pred,labels=['one','two','three','four','five']))

Random Forests Accuracy:  0.436
[[101  15  27  46  60]
 [ 43  17  36  72  66]
 [ 21  14  68 152 117]
 [ 14   7  63 265 431]
 [ 11   4  23 188 639]]
